In [1]:
GLOBAL_SEED = 7532

from numpy.random import seed
seed(GLOBAL_SEED)
from tensorflow import set_random_seed
set_random_seed(GLOBAL_SEED)

import numpy as np
import pandas as pd

from keras.models import Model, load_model, Sequential
from keras.callbacks import Callback, EarlyStopping, History, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Dense, Dropout, Conv2D, Flatten, Input, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input

from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata_HMGD.csv')
valid_set = pd.read_csv('valid_set_metadata_HMGD.csv')
test_set = pd.read_csv('test_set_metadata_HMGD.csv')

Initialize constants.

In [3]:
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0002
N_EPOCHS = 50
BATCH_SIZE = 32

Define the CNN model architecture.

In [4]:
xception_model = Xception(weights='imagenet', include_top=False, pooling=None, input_shape=INPUT_SHAPE)
xception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 159, 159, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 159, 159, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 159, 159, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Freeze the convolutional layers, then create the model top and attach it to the model.

In [5]:
for layer in xception_model.layers:
    layer.trainable = False

In [6]:
input_shape = xception_model.outputs[0].get_shape().as_list()[1:]

max_pool  = MaxPool2D(2)(xception_model.layers[-1].output)
batch_nrm = BatchNormalization()(max_pool)

flatten   = Flatten(input_shape=input_shape)(batch_nrm)
dropout_1 = Dropout(rate=0.5, seed=GLOBAL_SEED)(flatten)
dense_1   = Dense(512, activation='elu')(dropout_1)
dropout_2 = Dropout(rate=0.5, seed=GLOBAL_SEED)(dense_1)
dense_2   = Dense(256, activation='elu')(dropout_2)
dropout_3 = Dropout(rate=0.5, seed=GLOBAL_SEED)(dense_2)
model_top = Dense(1, activation='sigmoid', name='top_output')(dropout_3)

model = Model(inputs=xception_model.inputs, outputs=model_top)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 159, 159, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 159, 159, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 159, 159, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Compute the class weights to be used during model training in order to mitigate the class imbalances.

In [6]:
class_values = train_set['gender'].values
classes = np.unique(class_values)

weights = compute_class_weight(
    'balanced', 
    classes, 
    class_values
)
class_weights = dict(zip(classes, weights))
class_weights

{0: 1.0949726587842759, 1: 0.9201873755187876}

Train the model (via transfer learning). Note that this initial training is manually stopped after a certain number of epochs.

In [8]:
# Create and compile the model
adam = Adam(
    lr=LEARNING_RATE
)
model.compile(
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

# Initialize callbacks
checkpoint = ModelCheckpoint(
    'model_Xception_epoch_{epoch:03d}.hdf5', 
    monitor='val_loss', 
    save_best_only=False, 
    save_weights_only=False
)
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10
)    
callback_list = [checkpoint, lr_reduction, early_stopping]

# generate and augment training and validation data
data_generator = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=train_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)
valid_data_generator = data_generator.flow_from_dataframe(
    dataframe=valid_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

train_steps_per_epoch = train_data_generator.n//train_data_generator.batch_size
valid_steps_per_epoch = valid_data_generator.n//valid_data_generator.batch_size

#fit the model
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=train_steps_per_epoch, 
    epochs=N_EPOCHS,
    callbacks=callback_list,
    validation_data=valid_data_generator,
    validation_steps=valid_steps_per_epoch,
    class_weight=class_weights,
    workers=4, 
    verbose=1
)

Found 165400 images.
Found 4176 images.
Epoch 1/50
5168/5168 [==============================] - 3591s 695ms/step - loss: 0.5122 - binary_accuracy: 0.8041 - val_loss: 0.4811 - val_binary_accuracy: 0.8260
Epoch 2/50
5168/5168 [==============================] - 3567s 690ms/step - loss: 0.3659 - binary_accuracy: 0.8415 - val_loss: 0.4276 - val_binary_accuracy: 0.8318
Epoch 3/50
5168/5168 [==============================] - 3567s 690ms/step - loss: 0.3535 - binary_accuracy: 0.8488 - val_loss: 0.4471 - val_binary_accuracy: 0.8195
Epoch 4/50
5168/5168 [==============================] - 3751s 726ms/step - loss: 0.3490 - binary_accuracy: 0.8520 - val_loss: 0.4102 - val_binary_accuracy: 0.8292
Epoch 5/50
5168/5168 [==============================] - 3743s 724ms/step - loss: 0.3464 - binary_accuracy: 0.8531 - val_loss: 0.4124 - val_binary_accuracy: 0.8304
Epoch 6/50
5168/5168 [==============================] - 3913s 757ms/step - loss: 0.3443 - binary_accuracy: 0.8547 - val_loss: 0.4005 - val_binary

KeyboardInterrupt: 

Fine-tune the model.

In [4]:
model = load_model('model_Xception_epoch_011.hdf5')

for layer in model.layers[-56:]:
    layer.trainable = True

# compile the model to reflect the above changes
adam = Adam(
    lr=0.0001
)
model.compile(
    optimizer=adam, 
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 159, 159, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 159, 159, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 159, 159, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [7]:
# Initialize callbacks
checkpoint = ModelCheckpoint(
    'model_Xception_finetuning_epoch_{epoch:03d}.hdf5', 
    monitor='val_loss', 
    save_best_only=False, 
    save_weights_only=False
)
lr_reduction = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10
)    
callback_list = [checkpoint, lr_reduction, early_stopping]

# generate and augment training and validation data
data_generator = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)
train_data_generator = data_generator.flow_from_dataframe(
    dataframe=train_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)
valid_data_generator = data_generator.flow_from_dataframe(
    dataframe=valid_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=BATCH_SIZE,
    seed=GLOBAL_SEED
)

train_steps_per_epoch = train_data_generator.n//train_data_generator.batch_size
valid_steps_per_epoch = valid_data_generator.n//valid_data_generator.batch_size

#fit the model
history = model.fit_generator(
    generator=train_data_generator,
    steps_per_epoch=train_steps_per_epoch, 
    epochs=N_EPOCHS,
    callbacks=callback_list,
    validation_data=valid_data_generator,
    validation_steps=valid_steps_per_epoch,
    class_weight=class_weights,
    workers=4, 
    verbose=1
)

Found 165400 images.
Found 4176 images.
Epoch 1/50
5168/5168 [==============================] - 5511s 1s/step - loss: 0.2885 - binary_accuracy: 0.8857 - val_loss: 0.3271 - val_binary_accuracy: 0.8548
Epoch 2/50
5168/5168 [==============================] - 5496s 1s/step - loss: 0.2604 - binary_accuracy: 0.8981 - val_loss: 0.2890 - val_binary_accuracy: 0.8868
Epoch 3/50
5168/5168 [==============================] - 5491s 1s/step - loss: 0.2519 - binary_accuracy: 0.9021 - val_loss: 0.2905 - val_binary_accuracy: 0.8764
Epoch 4/50
5168/5168 [==============================] - 5494s 1s/step - loss: 0.2424 - binary_accuracy: 0.9052 - val_loss: 0.3026 - val_binary_accuracy: 0.8936
Epoch 5/50
5168/5168 [==============================] - 5492s 1s/step - loss: 0.2359 - binary_accuracy: 0.9067 - val_loss: 0.2728 - val_binary_accuracy: 0.8883
Epoch 6/50
5168/5168 [==============================] - 5486s 1s/step - loss: 0.2307 - binary_accuracy: 0.9081 - val_loss: 0.3069 - val_binary_accuracy: 0.8798


Evaluate the model on the test set.

In [9]:
evaluation_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_dataframe(
    dataframe=test_set, 
    directory='imdb_crop/_all_photos/',
    x_col='photo_path',
    y_col='gender',
    target_size=(320, 320),
    class_mode='other',
    batch_size=1,
    seed=GLOBAL_SEED,
    shuffle=False
)

best_model = load_model('model_Xception_finetuning_epoch_005.hdf5')
best_model.evaluate_generator(generator=evaluation_data_generator, steps=len(test_set))

Found 7621 images.


[0.2866261049134404, 0.88794121506364]